In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

In [ ]:

import os
import cv2
from time import time
import numpy as np
import pandas as pd
from yolov5.models.yolo import Model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torchvision


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DL Project/TrafficSign DataSet.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


In [ ]:
def ResizeImage(image: np.ndarray, newWidth: int, newHeight: int) -> tuple:
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    return resizedImage

In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/tafficSignAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Catoregy'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread("."+row[1])
        images.append(image)
        annotations.append(uniqueSigns.index(row[0]))
    del annotationsDataFrame
    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        label = annotations[i]
        resizedImage = ResizeImage(
            image, resize[0], resize[1])
        resizedImages.append(resizedImage)
        newAnnotations.append(
            [(i % batchSize), label])

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
def TrainModel(model, dataLoader, epochs, optimizer, criterion, device):
    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch+1, epochs))
        startTime = time()
        runningLoss = 0
        dataLoaderLen = len(dataLoader)
        runningAccuracy = 0

        for i, (inputs, targets) in enumerate(dataLoader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            runningLoss += loss.item() * inputs.size(0)
            runningAccuracy += torch.sum(preds == labels.data)
            if(((i*100)//dataLoaderLen) % 10 == 0):
                print((i*100//dataLoaderLen), end="%,")
        endTime = time()
        timeTaken = endTime-startTime
        epochLoss = runningLoss / dataLoaderLen
        epochAccuracy = runningAccuracy.double() / dataLoaderLen
        print()
        print("Training Loss: {:.4f}".format(epochLoss))
        print("Training Accuracy: {:.4f}".format(epochAccuracy))
        print("Time taken: {}min, {}, secs".format(timeTaken//60, timeTaken % 60))


    print('Training complete')
    return model


In [ ]:
batchSize = 32
inputShape = (416, 416)
epochs = 100
numAnchors = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print("Using {} device".format(device))


In [ ]:
images, annotations, numClasses = LoadDataSet("./TrafficSign DataSet")


In [ ]:
X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations


In [ ]:
trainDataLoader, valDataLoader = CreateDataLoaders(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val


In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, numClasses)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
trainedModel = TrainModel(model, trainDataLoader, epochs, optimizer, criterion, device)

In [ ]:
torch.save(trainedModel.state_dict(), 'signPredictionV1.pth')
trainedModel.eval()
